# Data Preprocessing
:label:`sec_pandas`

So far, we have been working with synthetic data
that arrived in ready-made tensors.
However, to apply deep learning in the wild
we must extract messy data
stored in arbitrary formats,
and preprocess it to suit our needs.
Fortunately, the *pandas* [library](https://pandas.pydata.org/)
can do much of the heavy lifting.
This section, while no substitute
for a proper *pandas* [tutorial](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html),
will give you a crash course
on some of the most common routines.

## Reading the Dataset

Comma-separated values (CSV) files are ubiquitous
for the storing of tabular (spreadsheet-like) data.
In them, each line corresponds to one record
and consists of several (comma-separated) fields, e.g.,
"Albert Einstein,March 14 1879,Ulm,Federal polytechnic school,field of gravitational physics".
To demonstrate how to load CSV files with `pandas`,
we (**create a CSV file below**) `../data/house_tiny.csv`.
This file represents a dataset of homes,
where each row corresponds to a distinct home
and the columns correspond to the number of rooms (`NumRooms`),
the roof type (`RoofType`), and the price (`Price`).


In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')

Now let's import `pandas` and load the dataset with `read_csv`.


In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000


## Data Preparation

In supervised learning, we train models
to predict a designated *target* value,
given some set of *input* values.
Our first step in processing the dataset
is to separate out columns corresponding
to input versus target values.
We can select columns either by name or
via integer-location based indexing (`iloc`).

Here are some common imputation heuristics.
[**For categorical input fields,
we can treat `NaN` as a category.**]
Since the `RoofType` column takes values `Slate` and `NaN`,
`pandas` can convert this column
into two columns `RoofType_Slate` and `RoofType_nan`.
A row whose roof type is `Slate` will set values
of `RoofType_Slate` and `RoofType_nan` to 1 and 0, respectively.
The converse holds for a row with a missing `RoofType` value.


In [3]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       NaN           False          True
1       2.0           False          True
2       4.0            True         False
3       NaN           False          True


For missing numerical values,
one common heuristic is to
[**replace the `NaN` entries with
the mean value of the corresponding column**].


In [4]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       3.0           False          True
1       2.0           False          True
2       4.0            True         False
3       3.0           False          True


## Conversion to the Tensor Format

Now that [**all the entries in `inputs` and `targets` are numerical,
we can load them into a tensor**] (recall :numref:`sec_ndarray`).


In [5]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(targets.to_numpy(dtype=float))
X, y

(tensor([[3., 0., 1.],
         [2., 0., 1.],
         [4., 1., 0.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## Discussion

You now know how to partition data columns,
impute missing variables,
and load `pandas` data into tensors.
In :numref:`sec_kaggle_house`, you will
pick up some more data processing skills.
While this crash course kept things simple,
data processing can get hairy.
For example, rather than arriving in a single CSV file,
our dataset might be spread across multiple files
extracted from a relational database.
For instance, in an e-commerce application,
customer addresses might live in one table
and purchase data in another.
Moreover, practitioners face myriad data types
beyond categorical and numeric, for example,
text strings, images,
audio data, and point clouds.
Oftentimes, advanced tools and efficient algorithms
are required in order to prevent data processing from becoming
the biggest bottleneck in the machine learning pipeline.
These problems will arise when we get to
computer vision and natural language processing.
Finally, we must pay attention to data quality.
Real-world datasets are often plagued
by outliers, faulty measurements from sensors, and recording errors,
which must be addressed before
feeding the data into any model.
Data visualization tools such as [seaborn](https://seaborn.pydata.org/),
[Bokeh](https://docs.bokeh.org/), or [matplotlib](https://matplotlib.org/)
can help you to manually inspect the data
and develop intuitions about
the type of problems you may need to address.


## Exercises

1. Try loading datasets, e.g., Abalone from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) and inspect their properties. What fraction of them has missing values? What fraction of the variables is numerical, categorical, or text?
1. Try indexing and selecting data columns by name rather than by column number. The pandas documentation on [indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) has further details on how to do this.
1. How large a dataset do you think you could load this way? What might be the limitations? Hint: consider the time to read the data, representation, processing, and memory footprint. Try this out on your laptop. What happens if you try it out on a server?
1. How would you deal with data that has a very large number of categories? What if the category labels are all unique? Should you include the latter?
1. What alternatives to pandas can you think of? How about [loading NumPy tensors from a file](https://numpy.org/doc/stable/reference/generated/numpy.load.html)? Check out [Pillow](https://python-pillow.org/), the Python Imaging Library.


In [6]:
pip install ucimlrepo

In [7]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
census_income = fetch_ucirepo(id=20)

# data (as pandas dataframes)
X = census_income.data.features
y = census_income.data.targets

# metadata
print(census_income.metadata)

# variable information
print(census_income.variables)


{'uci_id': 20, 'name': 'Census Income', 'repository_url': 'https://archive.ics.uci.edu/dataset/20/census+income', 'data_url': 'https://archive.ics.uci.edu/static/public/20/data.csv', 'abstract': 'Predict whether income exceeds $50K/yr based on census data.  Also known as Adult dataset.', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Mon Sep 09 2024', 'dataset_doi': '10.24432/C5GP7S', 'creators': ['Ron Kohavi'], 'intro_paper': None, 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && 

In [18]:
# Extract features and metadata
X = census_income.data.features
metadata = census_income.variables
print(X.columns)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')


In [10]:
# Determine columns with missing values
missing_columns = metadata[metadata['missing_values'] == 'yes']
missing_fraction = len(missing_columns) / len(metadata)
print(f"Fraction of columns with missing values: {missing_fraction}")

Fraction of columns with missing values: 0.2


In [11]:
# Determine the count of numerical, categorical, and binary columns
num_numerical = len(metadata[metadata['type'] == 'Integer'])
num_categorical = len(metadata[metadata['type'] == 'Categorical'])
num_binary = len(metadata[metadata['type'] == 'Binary'])
print(f"Number of numerical columns: {num_numerical}")
print(f"Number of categorical columns: {num_categorical}")
print(f"Number of binary columns: {num_binary}")

Number of numerical columns: 6
Number of categorical columns: 7
Number of binary columns: 2


In [14]:
# Select the 'age' column
age_column = X['age']

# Selecting 'age' and 'workclass' columns
selected_columns = X[['age', 'workclass']]

# Display the first few rows of the selected columns
print(selected_columns.head())

   age         workclass
0   39         State-gov
1   50  Self-emp-not-inc
2   38           Private
3   53           Private
4   28           Private


In [15]:
# Selecting 'age' and 'education' columns
selected_columns = X.loc[:, ['age', 'education']]

In [21]:
# Filter rows in X where age is greater than 40
filtered_features = X.loc[X['age'] > 40].copy()

# Retrieve the corresponding 'income' values from y based on the same condition
filtered_income = y.loc[X['age'] > 40]

# Add the 'income' column to the filtered features
# Change: Access the 'income' column of filtered_income as a Series
filtered_features['income'] = filtered_income['income']

# Display the result
print(filtered_features)


       age         workclass  fnlwgt  education  education-num  \
1       50  Self-emp-not-inc   83311  Bachelors             13   
3       53           Private  234721       11th              7   
6       49           Private  160187        9th              5   
7       52  Self-emp-not-inc  209642    HS-grad              9   
9       42           Private  159449  Bachelors             13   
...    ...               ...     ...        ...            ...   
48831   48           Private  285570    HS-grad              9   
48832   61           Private   89686    HS-grad              9   
48835   48         Local-gov  349230    Masters             14   
48838   64               NaN  321403    HS-grad              9   
48840   44           Private   83891  Bachelors             13   

              marital-status         occupation    relationship  \
1         Married-civ-spouse    Exec-managerial         Husband   
3         Married-civ-spouse  Handlers-cleaners         Husband   
6     

In [22]:
combined_df = pd.concat([X, y], axis=1)

# Filter based on 'age' and select 'age' and 'income' columns
filtered_data = combined_df.loc[combined_df['age'] > 40, ['age', 'income']]

# Display the result
print(filtered_data)

       age  income
1       50   <=50K
3       53   <=50K
6       49   <=50K
7       52    >50K
9       42    >50K
...    ...     ...
48831   48  <=50K.
48832   61  <=50K.
48835   48  <=50K.
48838   64  <=50K.
48840   44  <=50K.

[20211 rows x 2 columns]


[Discussions](https://discuss.d2l.ai/t/29)
